In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
cpp_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/extracted_data/cpp_df.csv'
ptio2_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/extracted_data/ptio2_df.csv'
drug_administration_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/drug_administrations.xlsx'

In [ ]:
cpp_df = pd.read_csv(cpp_path)
ptio2_df = pd.read_csv(ptio2_path)
drug_administration_df = pd.read_excel(drug_administration_path)

# Data Exploration

In [ ]:
drug_administration_df = drug_administration_df[drug_administration_df.monitored]

In [ ]:
cpp_df.head()

In [ ]:
ptio2_df.head()

In [ ]:
drug_administration_df.head()

In [ ]:
lower_bound

In [ ]:
# for every drug administration extract data from -12h to +12h around start
associated_ptio2_df = pd.DataFrame()
associated_cpp_df = pd.DataFrame()
ptio2_df['datetime'] = pd.to_datetime(ptio2_df['datetime'])
cpp_df['datetime'] = pd.to_datetime(cpp_df['datetime'])

for index, row in drug_administration_df.iterrows():
    lower_bound = row['drug_start'] - pd.to_timedelta(12, unit='h')
    upper_bound = row['drug_start'] + pd.to_timedelta(12, unit='h')
    instance_associated_ptio2_df = ptio2_df[(ptio2_df['pat_nr'] == row['pat_nr'])
                                            & (ptio2_df['datetime'] >= lower_bound) 
                                            & (ptio2_df['datetime'] <= upper_bound)]
    instance_associated_ptio2_df['drug_start'] = row['drug_start']
    instance_associated_ptio2_df['relative_datetime'] = (instance_associated_ptio2_df['datetime'] - row['drug_start']).dt.total_seconds() / 3600
    associated_ptio2_df = pd.concat([associated_ptio2_df, instance_associated_ptio2_df])
    
    instance_associated_cpp_df = cpp_df[(cpp_df['pat_nr'] == row['pat_nr'])
                                        & (cpp_df['datetime'] >= lower_bound) 
                                        & (cpp_df['datetime'] <= upper_bound)]  
    instance_associated_cpp_df['drug_start'] = row['drug_start']
    instance_associated_cpp_df['relative_datetime'] = (instance_associated_cpp_df['datetime'] - row['drug_start']).dt.total_seconds() / 3600
    associated_cpp_df = pd.concat([associated_cpp_df, instance_associated_cpp_df])
    

In [ ]:
associated_ptio2_df.head()

In [ ]:
associated_cpp_df.head()

# Data Cleaning

In [ ]:
allowed_ptio2_range = [0, 200]
allowed_cpp_range = [0, 200]

In [ ]:
# drop values outside of allowed range
associated_ptio2_df = associated_ptio2_df[(associated_ptio2_df['ptio2'] >= allowed_ptio2_range[0]) & (associated_ptio2_df['ptio2'] <= allowed_ptio2_range[1])]
associated_cpp_df = associated_cpp_df[(associated_cpp_df['cpp'] >= allowed_cpp_range[0]) & (associated_cpp_df['cpp'] <= allowed_cpp_range[1])]

meta summary

In [ ]:
drug_administration_df.pat_nr.nunique(), drug_administration_df.shape

# Data Plotting

In [ ]:
associated_ptio2_df['rounded_relative_datetime'] = associated_ptio2_df['relative_datetime'].round(1)
associated_cpp_df['rounded_relative_datetime'] = associated_cpp_df['relative_datetime'].round(1)

In [ ]:
# plot a line plot of ptio2 and cpp values over relative time

fig, ax = plt.subplots(1, 1, figsize=(20, 10))
sns.lineplot(x='rounded_relative_datetime', y='ptio2', data=associated_ptio2_df, ax=ax, label='PtiO2', legend=False)
# create second y axis for cpp values
ax2 = ax.twinx()
sns.lineplot(x='rounded_relative_datetime', y='cpp', data=associated_cpp_df, ax=ax2, color='magenta', label='CPP', legend=False)

ax.set_xlabel('Relative time from administration (h)')
ax.set_ylabel('PtiO2 (mmHg)')
ax2.set_ylabel('CPP (mmHg)')

ax2.set_ylim(60,100)

# show legend
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax.legend(lines + lines2, labels + labels2, loc='upper right')

# add vertical line at x=0 with text label with "Diclofenac administration"
ax.axvline(x=0, ymin=0.015, ymax=0.1, color='black', linestyle='--', linewidth=0.7)
ax.text(0.5, 0.12, 'Diclofenac 75mg', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)


In [ ]:
# fig.savefig('/Users/jk1/Downloads/ptio2_cpp_plot.png')

In [ ]:
# TODO: add temperature curve